In [1]:
from gurobipy import *
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt

In [33]:
output_file = 'variable_values'

results = pd.read_table(f'../output/{output_file}.txt')
n = results[results["results"].str.contains("n")]
u = results[results["results"].str.contains("u")]
c = results[results["results"].str.contains("c")]
varn = n['results'].str.split('=').apply(lambda x: x[0])
valn = n['results'].str.split('=').apply(lambda x: x[1])
t = varn.str.split('n').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[0])
i = varn.str.split('n').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[1])
k = varn.str.split('n').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[2])
amount = valn
specificn = pd.DataFrame(np.array([t,i,k,amount]).T).reset_index(drop=True)
specificn.columns = ['t','i', 'k', 'amount']
specificn['name'] = 'n'

varu = u['results'].str.split('=').apply(lambda x: x[0])
valu = u['results'].str.split('=').apply(lambda x: x[1])
t = varu.str.split('u').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[0])
i = varu.str.split('u').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[1])
j = varu.str.split('u').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[2])
k = varu.str.split('u').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[3])
amount = valu
specificu = pd.DataFrame(np.array([t,i,j,k,amount]).T).reset_index(drop=True)
specificu.columns = ['t','i', 'j', 'k', 'amount']
specificu['name'] = 'u'

varc = c['results'].str.split('=').apply(lambda x: x[0])
valc = c['results'].str.split('=').apply(lambda x: x[1])
t = varc.str.split('c').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[0])
i = varc.str.split('c').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[1])
x = varc.str.split('c').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[2])
y = varc.str.split('c').apply(lambda x: ast.literal_eval(x[1])).apply(lambda x: x[3])
amount = valc
specificc = pd.DataFrame(np.array([t,i,x,y,amount]).T).reset_index(drop=True)
specificc.columns = ['t','i', 'x', 'y', 'amount']
specificc['name'] = 'c'
specificn['amount'] = specificn['amount'].astype(float)
specificu['amount'] = specificu['amount'].astype(float)
specificc['amount'] = specificc['amount'].astype(float)


In [34]:
specificc

,t,i,x,y,amount,name
0,0,1,0,2,1.0,c
1,0,1,0,5,1.0,c
2,0,1,2,4,1.0,c
3,1,0,0,2,2.0,c
4,1,0,2,3,1.0,c
...,...,...,...,...,...,...
515,288,0,0,2,2.0,c
516,288,1,0,2,1.0,c
517,290,1,0,2,1.0,c
518,290,1,0,5,1.0,c


In [35]:
specificn

,t,i,k,amount,name
0,0,0,0,2.0,n
1,0,0,2,2.0,n
2,0,0,5,1.0,n
3,0,1,1,1.0,n
4,0,1,4,1.0,n
...,...,...,...,...,...
102,290,0,2,2.0,n
103,290,0,5,1.0,n
104,290,1,1,1.0,n
105,290,1,4,1.0,n


In [36]:
specificu01 = specificu[specificu['i'] == 0]
specificu10 = specificu[specificu['i'] == 1]

In [37]:
specificu01

,t,i,j,k,amount,name
0,1,0,1,5,1.0,u
2,2,0,1,3,1.0,u
4,3,0,1,3,1.0,u
6,4,0,1,1,1.0,u
9,5,0,1,3,2.0,u
...,...,...,...,...,...,...
752,284,0,1,2,1.0,u
754,285,0,1,1,2.0,u
757,286,0,1,1,1.0,u
759,287,0,1,1,2.0,u


In [38]:
soc_level = {
    0: 20,
    1: 30,
    2: 40,
    3: 50,
    4: 60,
    5: 70,
    6: 80,
    7: 90,
    8: 100
}

In [39]:
gamma = [3.237872699, 3.729470167, 4.404786588, 5.379957014, 6.913363091,
        9.685271742, 16.30528373, 71.41103553]
gamma = np.array(gamma) / 5
gamma = np.ceil(gamma)

In [48]:
# def extract_day(df, start, end):
#     return df.iloc[start:end].reset_index(drop=True)

# def extract_day_from_t(df, start, end):
#     return df[(df['t'] >= start) & (df['t'] <= end)].reset_index(drop=True)

def rename_columns(df):
    df.columns = ['t', 'num_flights', 'aircraft_pushback_time']
    return df

def convert_timestep_to_sec(df, timestep=5):
    df['aircraft_pushback_time'] = df.loc[:, 't'] * timestep * 60
    return df

def replicate_rows(df, column_name):
    return df.loc[df.index.repeat(df[column_name])].reset_index(drop=True)

def reorganize_df(df, timestep=5):
    df = convert_timestep_to_sec(df, timestep)
    df = replicate_rows(df, 'amount')
    # df = rename_columns(df)
    return df

def get_charge_time(row):
    start = max(row['x']-1, 0)
    end = row['y']
    return sum(gamma[start:end])*60*5

def create_location(df, location_col):
    df['location'] = df.groupby(location_col).cumcount() + 11
    df['location'] = df[location_col] + '_PARK' + df['location'].astype(str)
    df = none_location(df)
    return df

# add location if the charging_start_time is 0, otherwise add None
def none_location(df):
    df['location'] = np.where(df['charging_start_time'] == 0, df['location'], None)
    return df

In [41]:
df_c = specificc
df_c['charging_start_time'] = df_c['t'] * 5 * 60
# df_c['charging_time'] = df_c.apply(get_charge_time, axis=1)
df_c['i'] = df_c['i'].apply(lambda x: 'LAX' if x == 0 else 'DTLA')
df_c.rename(columns={'amount': 'num_aircraft', 
                     'x': 'init_soc',
                     'y': 'target_soc',
                     'i': 'vertiport_id'}, inplace=True)
df_c['init_soc'] = df_c['init_soc'].apply(lambda x: soc_level[x])
df_c['target_soc'] = df_c['target_soc'].apply(lambda x: soc_level[x])
# Duplicate rows based on num_aircraft column
# df_c = df_c.loc[df_c.index.repeat(df_c.num_aircraft)].reset_index(drop=True)
df_c.drop(['t','name', 'num_aircraft'], axis=1, inplace=True)
df_c['process'] = 'CHARGING'
df_c

,vertiport_id,init_soc,target_soc,charging_start_time,process
0,DTLA,20,40,0,CHARGING
1,DTLA,20,70,0,CHARGING
2,DTLA,40,60,0,CHARGING
3,LAX,20,40,300,CHARGING
4,LAX,40,50,300,CHARGING
...,...,...,...,...,...
515,LAX,20,40,86400,CHARGING
516,DTLA,20,40,86400,CHARGING
517,DTLA,20,40,87000,CHARGING
518,DTLA,20,70,87000,CHARGING


In [54]:
def get_idling_aircraft(df):
    df = reorganize_df(df)
    # df = df.loc[df.index.repeat(df.num_flights)].reset_index(drop=True)
    df['vertiport_id'] = df['i'].apply(lambda x: 'LAX' if x == 0 else 'DTLA')
    df = df[df['aircraft_pushback_time'] == 0]
    df.drop(['t', 'i', 'name', 'amount', 'aircraft_pushback_time'], axis=1, inplace=True)
    df.rename(columns={'k': 'soc'}, inplace=True)
    df['init_soc'] = df['soc'].apply(lambda x: soc_level[x])
    df['target_soc'] = df['soc'].apply(lambda x: soc_level[x])
    df['charging_start_time'] = 0
    df.drop('soc', axis=1, inplace=True)
    df['process'] = 'PARKING' 
    return df

In [55]:
idle_aircraft = get_idling_aircraft(specificn)
idle_aircraft

,vertiport_id,init_soc,target_soc,charging_start_time,process
0,LAX,20,20,0,PARKING
1,LAX,20,20,0,PARKING
2,LAX,40,40,0,PARKING
3,LAX,40,40,0,PARKING
4,LAX,70,70,0,PARKING
5,DTLA,30,30,0,PARKING
6,DTLA,60,60,0,PARKING
7,DTLA,70,70,0,PARKING
8,DTLA,70,70,0,PARKING


In [58]:
init_states = pd.concat([df_c, idle_aircraft]).sort_values(['charging_start_time']).reset_index(drop=True)


create_location(init_states, 'vertiport_id')

,vertiport_id,init_soc,target_soc,charging_start_time,process,location
0,DTLA,20,40,0,CHARGING,DTLA_PARK11
1,DTLA,60,60,0,PARKING,DTLA_PARK12
2,DTLA,30,30,0,PARKING,DTLA_PARK13
3,LAX,70,70,0,PARKING,LAX_PARK11
4,LAX,40,40,0,PARKING,LAX_PARK12
...,...,...,...,...,...,...
524,DTLA,20,40,86400,CHARGING,None
525,LAX,20,40,86400,CHARGING,None
526,DTLA,20,40,87000,CHARGING,None
527,DTLA,20,70,87000,CHARGING,None


In [65]:
def create_flight_schedule(df, origin, destination):
    df = reorganize_df(df)
    # df = df.loc[df.index.repeat(df.num_flights)].reset_index(drop=True)
    df['origin_vertiport_id'] = origin
    df['destination_vertiport_id'] = destination
    df['aircraft_pushback_time'] = df['aircraft_pushback_time'] + df.groupby('aircraft_pushback_time').cumcount() * 30
    df.drop(['t', 'i', 'j', 'name', 'amount'], axis=1, inplace=True)
    df.rename(columns={'k': 'soc'}, inplace=True)
    df['soc'] = df['soc'].apply(lambda x: soc_level[x])
    df = df[['origin_vertiport_id', 'destination_vertiport_id', 'aircraft_pushback_time', 'soc']]
    df['tail_number'] = None
    return df

In [66]:
lax_dtla = create_flight_schedule(specificu01, origin='LAX', destination='DTLA')
dtla_lax = create_flight_schedule(specificu10, origin='DTLA', destination='LAX')

/var/folders/f3/kcmw8dgs2yj5rlxtpqsq4nbc0000gn/T/ipykernel_2160/3630703081.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['aircraft_pushback_time'] = df.loc[:, 't'] * timestep * 60
/var/folders/f3/kcmw8dgs2yj5rlxtpqsq4nbc0000gn/T/ipykernel_2160/3630703081.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['aircraft_pushback_time'] = df.loc[:, 't'] * timestep * 60


In [68]:
flight_schedule = pd.concat([lax_dtla, dtla_lax]).sort_values('aircraft_pushback_time').reset_index(drop=True)
flight_schedule

,origin_vertiport_id,destination_vertiport_id,aircraft_pushback_time,soc,tail_number
0,LAX,DTLA,300,70,None
1,DTLA,LAX,300,60,None
2,LAX,DTLA,600,50,None
3,DTLA,LAX,600,40,None
4,LAX,DTLA,900,50,None
...,...,...,...,...,...
961,DTLA,LAX,86100,80,None
962,LAX,DTLA,86130,30,None
963,LAX,DTLA,86400,40,None
964,DTLA,LAX,86400,30,None


In [69]:
flight_schedule.to_csv('../output/flight_schedule.csv', index=False)